# Analisis de correlacion

In [2]:
import pandas as pd
import plotly.express as px
import os

In [3]:
# Configurar colores
COLORES = {
    'Principal': '#2E4053',
    'Secundario': '#1ABC9C',
    'Alerta': '#E74C3C',
    'Neutro': '#95A5A6'
}

In [4]:
# --- 1. CARGA DE DATOS ---
ruta_datos = os.path.join("..", "data", "processed", "telecom_final.csv")


In [5]:
# Solo para verificar que el archivo existe y se puede cargar
try:
    df = pd.read_csv(ruta_datos)
    print(f"Datos cargados para análisis avanzado: {df.shape[0]} registros.")
except FileNotFoundError:
    print("Error: No se encontró el archivo. Verifica los pasos anteriores.")


Datos cargados para análisis avanzado: 7267 registros.


### INGENIERÍA DE VARIABLES 

Desafío: ¿Cómo afecta la CANTIDAD de servicios contratados a la evasión?
Para esto, necesitamos contar cuántos servicios tiene activos cada cliente.

In [6]:
# Nota: Algunas son 1/0 y otras son Texto ('Yes', 'No', 'Sin servicio')
cols_servicios = [
    'Servicio_Telefonico', 'Lineas_Multiples', 'Seguridad_Online',
    'Respaldo_Online', 'Proteccion_Dispositivo', 'Soporte_Tecnico',
    'Streaming_TV', 'Streaming_Peliculas'
]


In [8]:
# Función para contar: Sumamos 1 si es 'Yes' o el número 1
def contar_servicios(fila):
    contador = 0
    for col in cols_servicios:
        valor = fila[col]
        if valor == 1 or valor == 'Yes':
            contador += 1
    if fila['Servicio_Internet'] not in ['No', 'Sin servicio']:
        contador += 1
    return contador


df['Total_Servicios'] = df.apply(contar_servicios, axis=1)
display(df[['Total_Servicios', 'Abandono']].head())



,Total_Servicios,Abandono
0,5,0.0
1,4,0.0
2,3,1.0
3,6,1.0
4,4,1.0


### MATRIZ DE CORRELACIÓN 
Desafío: Usar corr() para ver qué factores numéricos están más ligados al Abandono.

In [10]:
cols_corr = ['Meses_Contrato', 'Cargo_Mensual', 'Gasto_Diario', 'Total_Servicios', 'Abandono']
matriz_corr = df[cols_corr].corr()

fig_corr = px.imshow(matriz_corr,
                     text_auto=".2f",
                     aspect="auto",
                     color_continuous_scale="RdBu_r", # Rojo = Correlación Positiva con Abandono
                     title="Mapa de Calor: ¿Qué correlaciona con la Fuga?")

fig_corr.show()

### VISUALIZACIÓN: SCATTER PLOT Total Gastado vs Antigüedad
Desafío: Gráfico de Dispersión.
Vamos a ver el Valor de Vida del Cliente (LTV).


In [12]:
fig_scatter = px.scatter(df, x='Meses_Contrato', y='Cargo_Total',
                         color='Abandono',
                         title='Valor del Cliente: Antigüedad vs Gasto Total',
                         color_discrete_map={0: COLORES['Secundario'], 1: COLORES['Alerta']},
                         opacity=0.6) 

fig_scatter.update_layout(
    xaxis_title="Meses de Antigüedad",
    yaxis_title="Total Gastado Histórico ($)"
)

fig_scatter.show()

### ANÁLISIS: CANTIDAD DE SERVICIOS VS EVASIÓN 
¿Tener más servicios hace que el cliente sea más fiel o que huya por el precio?


In [14]:
fig_bar_servicios = px.histogram(df, x='Total_Servicios', 
                                 color='Abandono',
                                 barmode='group',
                                 title='Evasión según Cantidad de Servicios Contratados',
                                 text_auto=True,
                                 color_discrete_map={0: COLORES['Secundario'], 1: COLORES['Alerta']})

fig_bar_servicios.update_layout(
    xaxis_title="Número de Servicios Activos",
    yaxis_title="Cantidad de Clientes",
    bargap=0.2
)

fig_bar_servicios.show()